#                                                                                     Lab n°3

1. Turn the dataset into a dataset compatible with Fastext (see the Tips on using FastText section a bit lower)

In [1]:
from datasets import load_dataset

train_dataset = load_dataset("imdb", split="train").to_pandas()
test_dataset = load_dataset("imdb", split="test").to_pandas()

test_dataset

Reusing dataset imdb (C:\Users\matth\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Reusing dataset imdb (C:\Users\matth\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0
...,...,...
24995,Just got around to seeing Monster Man yesterda...,1
24996,I got this as part of a competition prize. I w...,1
24997,I got Monster Man in a box set of three films ...,1
24998,"Five minutes in, i started to feel how naff th...",1


In [2]:
train_dataset['text'] = train_dataset['text'].str.lower()
test_dataset['text'] = test_dataset['text'].str.lower()

In [3]:
from string import punctuation

train_dataset['text'] = train_dataset['text'].str.replace('[{}]'.format(punctuation), '')
test_dataset['text'] = test_dataset['text'].str.replace('[{}]'.format(punctuation), '')

C:\Users\matth\AppData\Local\Temp\ipykernel_22620\1553221715.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train_dataset['text'] = train_dataset['text'].str.replace('[{}]'.format(punctuation), '')
C:\Users\matth\AppData\Local\Temp\ipykernel_22620\1553221715.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  test_dataset['text'] = test_dataset['text'].str.replace('[{}]'.format(punctuation), '')


In [4]:
test_dataset

,text,label
0,i love scifi and am willing to put up with a l...,0
1,worth the entertainment value of a rental espe...,0
2,its a totally average film with a few semialri...,0
3,star rating saturday night friday night fri...,0
4,first off let me say if you havent enjoyed a v...,0
...,...,...
24995,just got around to seeing monster man yesterda...,1
24996,i got this as part of a competition prize i wa...,1
24997,i got monster man in a box set of three films ...,1
24998,five minutes in i started to feel how naff thi...,1


In [5]:
def panda_to_fasttext(file, dataframe):
    '''Takes in a txt file path and a panda dataframe, write each dataframe row in the txt'''
    with open(file, 'w') as f:
        for index, row in dataframe.iterrows():
            f.writelines(f'__label__{str(row["label"]).encode("utf-8")} {row["text"].encode("utf-8")}\n')

In [6]:
panda_to_fasttext('imdb_train_comments.txt', train_dataset)

In [7]:
panda_to_fasttext('imdb_test_comments.txt', test_dataset)

In [8]:
!pip install fasttext

2. Train a FastText classifier with default parameters on the training data, and evaluate it on the test data using accuracy.

In [9]:
import fasttext

model = fasttext.train_supervised('imdb_train_comments.txt')

In [10]:
model.test('imdb_test_comments.txt')  

(25000, 0.87392, 0.87392)

3. Use the hyperparameters search functionality of FastText and repeat step 2

In [11]:
from sklearn.model_selection import train_test_split

train, validation = train_test_split(train_dataset, test_size=0.2, random_state=42, shuffle=True)

In [12]:
panda_to_fasttext('imdb_validation_comments.txt', validation)
panda_to_fasttext('imdb_train_comments.txt', train)

In [13]:
model = fasttext.train_supervised(input='imdb_train_comments.txt', autotuneValidationFile='imdb_validation_comments.txt')

In [14]:
model.test('imdb_test_comments.txt')

(25000, 0.88636, 0.88636)

4. Look at the differences between the default model and the attributes found with hyperparameters search. How do the two models differ?

The second model will take the best hyperparameters with the best f1 score. That's why the second model is always better.

5. Using the tuned model, take at least 2 wrongly classified examples from the test set, and try explaining why the model failed.

In [15]:
def predictions(data, model):
    '''Takes in a panda dataframe data and a fasttext model, returns and print a list of predictions'''
    preds = []
    for index, row in data.iterrows():
        pred = model.predict(row["text"])
        preds.append(pred)
        print("prediction : " + str(pred) + "    expected : " + str(row["label"]))
    return preds

preds = predictions(test_dataset, model)

prediction : (("__label__b'0'",), array([0.99732149]))    expected : 0
prediction : (("__label__b'0'",), array([0.9180463]))    expected : 0
prediction : (("__label__b'0'",), array([0.98925889]))    expected : 0
prediction : (("__label__b'0'",), array([0.97244519]))    expected : 0
prediction : (("__label__b'1'",), array([0.99960345]))    expected : 0
prediction : (("__label__b'0'",), array([0.98150927]))    expected : 0
prediction : (("__label__b'0'",), array([0.75001514]))    expected : 0
prediction : (("__label__b'0'",), array([0.99929881]))    expected : 0
prediction : (("__label__b'0'",), array([0.99894911]))    expected : 0
prediction : (("__label__b'0'",), array([1.00001001]))    expected : 0
prediction : (("__label__b'0'",), array([1.00000477]))    expected : 0
prediction : (("__label__b'0'",), array([0.97723174]))    expected : 0
prediction : (("__label__b'0'",), array([0.9999814]))    expected : 0
prediction : (("__label__b'0'",), array([0.99964333]))    expected : 0
predicti

In [16]:
print(test_dataset.iloc[4]["text"] + "\n")
print(test_dataset.iloc[18]["text"])

first off let me say if you havent enjoyed a van damme movie since bloodsport you probably will not like this movie most of these movies may not have the best plots or best actors but i enjoy these kinds of movies for what they are this movie is much better than any of the movies the other action guys segal and dolph have thought about putting out the past few years van damme is good in the movie the movie is only worth watching to van damme fans it is not as good as wake of death which i highly recommend to anyone of likes van damme or in hell but in my opinion its worth watching it has the same type of feel to it as nowhere to run good fun stuff

ben rupert grint is a deeply unhappy adolescent the son of his unhappily married parents his father nicholas farrell is a vicar and his mother laura linney is  well lets just say shes a somewhat hypocritical soldier in jesus army its only when he takes a summer job as an assistant to a foulmouthed eccentric oncefamous and nowforgotten actres

The reason why the model did not classify well the two examples is because the autors are using negative words in their sentences but to give a good opinion of the movie. For example the first author is using the double negation.

6. (Bonus point) Why is it likely that the attributes minn and maxn are at 0 after an hyperparameter search on our data ?